In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

# Charger les données
df = pd.read_csv('dvf_loyers.csv', sep=';')

# Nettoyer les données
df = df.dropna(subset=['valeur_fonciere', 'surface_reelle_bati', 'loyer_ref_m2'])
df = df[df['surface_reelle_bati'] > 0]
df = df[df['loyer_ref_m2'] > 0]

# Créer la liste des biens pour le dropdown
df['bien_label'] = df.apply(
    lambda row: f"{row['adresse_nom_voie']}, {row['nom_commune']} - {row['valeur_fonciere']:,.0f}€ - {row['surface_reelle_bati']:.0f}m²",
    axis=1
)

# Widgets pour les paramètres
bien_selector = widgets.Dropdown(
    options=list(zip(df['bien_label'], df.index)),
    description='Bien:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='90%')
)

apport_slider = widgets.IntSlider(
    value=50000,
    min=0,
    max=500000,
    step=5000,
    description='Apport (€):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='80%')
)

taux_slider = widgets.FloatSlider(
    value=3.5,
    min=0.5,
    max=6.0,
    step=0.1,
    description='Taux prêt (%):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='80%')
)

duree_slider = widgets.IntSlider(
    value=25,
    min=5,
    max=30,
    step=1,
    description='Durée (années):',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='80%')
)

output = widgets.Output()

def calculer_mensualite(capital, taux_annuel, duree_annees):
    """Calcule la mensualité d'un prêt"""
    if capital <= 0:
        return 0
    taux_mensuel = taux_annuel / 100 / 12
    nb_mois = duree_annees * 12
    if taux_mensuel == 0:
        return capital / nb_mois
    mensualite = capital * (taux_mensuel * (1 + taux_mensuel)**nb_mois) / ((1 + taux_mensuel)**nb_mois - 1)
    return mensualite

def afficher_resultats(change=None):
    """Affiche les résultats de la simulation"""
    with output:
        clear_output(wait=True)
        
        # Récupérer le bien sélectionné
        idx = bien_selector.value
        bien = df.loc[idx]
        
        # Paramètres de financement
        prix_achat = bien['valeur_fonciere']
        apport = apport_slider.value
        taux = taux_slider.value
        duree = duree_slider.value
        
        # Calculs
        montant_emprunte = max(0, prix_achat - apport)
        mensualite_pret = calculer_mensualite(montant_emprunte, taux, duree)
        cout_total = mensualite_pret * duree * 12
        cout_credit = cout_total - montant_emprunte
        
        # Calculs des loyers
        surface = bien['surface_reelle_bati']
        loyer_estime = surface * bien['loyer_ref_m2']
        loyer_majore = surface * bien['loyer_majore_m2']
        loyer_minore = surface * bien['loyer_minore_m2']
        
        # Charges estimées (25% du loyer)
        charges_estimees = loyer_estime * 0.25
        
        # Cash-flow
        cashflow_net = loyer_estime - mensualite_pret - charges_estimees
        
        # Rendements
        rendement_brut = (loyer_estime * 12 / prix_achat) * 100
        rendement_net = ((loyer_estime - charges_estimees) * 12 / prix_achat) * 100
        
        # Affichage formaté
        print("\n" + "-"*80)
        print("FINANCEMENT")
        print("-"*80)
        print(f"Apport personnel:    {apport:>15,.0f} € ({apport/prix_achat*100:.1f}%)")
        print(f"Montant emprunté:    {montant_emprunte:>15,.0f} €")
        print(f"Coût total crédit:   {cout_total:>15,.0f} €")
        print(f"Mensualité:          {mensualite_pret:>15,.0f} €/mois")

        print("\n" + "-"*80)
        print("LOYERS ET RENTABILITE")
        print("-"*80)
        print(f"Loyer MINORE:        {loyer_minore:>15,.0f} €/mois")
        print(f"Loyer REFERENCE:     {loyer_estime:>15,.0f} €/mois")
        print(f"Loyer MAJORE:        {loyer_majore:>15,.0f} €/mois\n")

        print(f"Loyer estimé:        {loyer_estime:>15,.0f} €/mois")
        print(f"Loyer annuel:        {loyer_estime*12:>15,.0f} €")
        print(f"Loyer/m²:            {bien['loyer_ref_m2']:>15.2f} €")
        print(f"Charges estimées:    {charges_estimees:>15,.0f} €/mois (25%)")
        print(f"(a la charge du locataire)")
        
        cashflow_status = "POSITIF" if cashflow_net >= 0 else "NEGATIF"
        print(f"\nCash-flow net:       {cashflow_net:>15,.0f} €/mois ({cashflow_status})")
        
        print(f"\nRendement brut bien:      {rendement_brut:>15.2f} %")
        print(f"Rendement net:       {rendement_net:>15.2f} %")
        

# Observer les changements
bien_selector.observe(afficher_resultats, names='value')
apport_slider.observe(afficher_resultats, names='value')
taux_slider.observe(afficher_resultats, names='value')
duree_slider.observe(afficher_resultats, names='value')

# Interface
print("="*80)
print("SIMULATEUR D'INVESTISSEMENT IMMOBILIER".center(80))
print("Données DVF + Loyers de référence Paris".center(80))
print("="*80)
print()

display(bien_selector)
display(apport_slider)
display(taux_slider)
display(duree_slider)
display(output)

# Affichage initial
afficher_resultats()

                     SIMULATEUR D'INVESTISSEMENT IMMOBILIER                     
                    Données DVF + Loyers de référence Paris                     



Dropdown(description='Bien:', layout=Layout(width='90%'), options=(('PAS BASFOUR, Paris 2e Arrondissement - 17…

IntSlider(value=50000, description='Apport (€):', layout=Layout(width='80%'), max=500000, step=5000, style=Sli…

FloatSlider(value=3.5, description='Taux prêt (%):', layout=Layout(width='80%'), max=6.0, min=0.5, style=Slide…

IntSlider(value=25, description='Durée (années):', layout=Layout(width='80%'), max=30, min=5, style=SliderStyl…

Output()